In [1]:
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from preprocess import merge_main_npy
import pickle

In [2]:
base_dir = "/home/hoseung/Work/data/BBS/"

action = 13
main_list = f"whoismain/{action}/main_list_{action}.txt"
npy_list = merge_main_npy(base_dir+main_list, prefix=base_dir + "npy_a/")

# 스켈레톤이 아주 stochastic하기 때문에 5장 중 median으로 smoothing

In [3]:
def smoothed_frame_sample(scene, fps = 1, window_size = 5):
    """일정 시간에 한번씩 ... 영상 길이가 다를 경우에 뒤를 0으로 채워야함 
    """
    FPS_ORIGINAL=10 # 인 것 같음...
    
    nskip = int(FPS_ORIGINAL/fps)
    nframe = np.ceil((len(scene) - window_size)/ nskip).astype(int)

    sub = np.zeros(nframe, dtype=scene.dtype) #scene.dtype - frame

    for i in range(nframe):
        temp = scene[i*nskip:i*nskip+window_size]

        for feat in temp.dtype.names: # recarry라서 한 번에 np.mean 불가능
            sub[i][feat] = np.median(temp[feat])

    return sub


def smoothed_frame_N(scene, nframe = 10, window_size = 5, shift=0):
    FPS_ORIGINAL=10 # 인 것 같음...
    
    nskip = int((len(scene)-shift) / nframe)
    #nframe = np.ceil((len(scene) - window_size)/ nskip).astype(int)

    sub = np.zeros(nframe, dtype=scene.dtype) #scene.dtype - frame

    for i in range(nframe):
        temp = scene[i*nskip+shift:i*nskip+window_size+shift]

        for feat in temp.dtype.names: # recarry라서 한 번에 np.mean 불가능
            sub[i][feat] = np.median(temp[feat])

    return sub


def ravel_rec(sub, return_feature=False):
    vec=[]
    if return_feature:
        features=[]
        for i, line in enumerate(sub):
            this_line=[]
            for ff in line.dtype.names:
                if ff != "frame":
                    this_line.append(line[ff])
                    features.append(ff+f"_{line['frame']}")
            vec.extend(this_line)
                    
        return features, np.array(vec)
    else:
        for line in sub:
            vec.extend([line[ff] for ff in line.dtype.names if ff not in "frame"])

    return np.array(vec)

In [4]:
scene = np.load(npy_list[0]['npy'])

In [7]:
scene

array([(  1, 314., 259., 303., 237., 295., 209., 298., 208., 300., 245., 309., 275., 316., 179., 296.5, 208.5, 294. , 263. , 288., 343., 305., 306., 291., 262., 297., 264., 300., 308., 288., 347.),
       (  2, 313., 262., 305., 239., 302., 209., 296., 209., 298., 244., 307., 274., 315., 179., 299. , 209. , 298. , 261.5, 293., 338., 304., 303., 300., 260., 296., 263., 296., 310., 287., 348.),
       (  3, 318., 262., 310., 239., 304., 209., 296., 209., 297., 245., 304., 274., 315., 178., 300. , 209. , 299. , 261.5, 294., 337., 304., 302., 301., 260., 297., 263., 297., 310., 288., 348.),
       (  4, 320., 262., 312., 238., 305., 209., 295., 209., 296., 245., 304., 274., 316., 178., 300. , 209. , 299. , 262. , 294., 336., 304., 302., 301., 260., 297., 264., 296., 310., 288., 349.),
       (  5, 320., 262., 312., 239., 305., 209., 296., 209., 296., 245., 304., 274., 316., 178., 300.5, 209. , 299. , 262. , 294., 336., 304., 302., 301., 260., 297., 264., 297., 310., 288., 348.),
       (  

In [6]:
scene.dtype

dtype([('frame', '<i8'), ('xl_hand', '<f8'), ('yl_hand', '<f8'), ('xl_elbow', '<f8'), ('yl_elbow', '<f8'), ('xl_shoulder', '<f8'), ('yl_shoulder', '<f8'), ('xr_shoulder', '<f8'), ('yr_shoulder', '<f8'), ('xr_elbow', '<f8'), ('yr_elbow', '<f8'), ('xr_hand', '<f8'), ('yr_hand', '<f8'), ('xhead', '<f8'), ('yhead', '<f8'), ('xneck', '<f8'), ('yneck', '<f8'), ('xpelvis', '<f8'), ('ypelvis', '<f8'), ('xl_foot', '<f8'), ('yl_foot', '<f8'), ('xl_knee', '<f8'), ('yl_knee', '<f8'), ('xl_hip', '<f8'), ('yl_hip', '<f8'), ('xr_hip', '<f8'), ('yr_hip', '<f8'), ('xr_knee', '<f8'), ('yr_knee', '<f8'), ('xr_foot', '<f8'), ('yr_foot', '<f8')])

In [104]:
scene = np.load(npy_list[0]['npy'])
nframe = 6
frame_skip = 20
data = []
label=[]
feature=[]

feature_ready=False
for i, fn_npy in enumerate(npy_list):
    if fn_npy['main'] in [0,1]:
        try:
        #if True:
            scene = np.load(fn_npy['npy'])
            for j in range(frame_skip):
                sub = smoothed_frame_N(scene, nframe=nframe, shift=j)
                
                # feature 기록
                if i == j == 0:
                    feature, rav_sub = ravel_rec(sub, return_feature=True)
                else:
                    rav_sub = ravel_rec(sub)
                data.append(rav_sub)
                label.append(fn_npy['score'])
                
        except:
            print("Main person was guessed, but not actually detected... ", fn_npy['npy'])


Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/1/003/a_003_13_1_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/1/057/a_057_13_1_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/1/061/a_061_13_1_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/4/062/a_062_13_4_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/1/088/a_088_13_1_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/3/125/a_125_13_3_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/0/291/a_291_13_0_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/0/331/a_331_13_0_1.npy


In [105]:
data = np.stack(data)
label = np.array(label)
print(data.shape)

(8180, 180)


In [106]:
from sklearn import preprocessing

sc = preprocessing.MinMaxScaler(feature_range=[0,1])

data = sc.fit_transform(data)

In [107]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(data, 
                                                                            label, 
                                                                            test_size=0.7,
                                                                            stratify=label)

In [120]:
small = True

if small:
    ntree = 20
    max_depth = 6
    fn_model_out = "./trained_model13_s.pickle"
    fn_data_out = "BBS_dataset_s.pickle"
else:
    ntree = 100
    max_depth = 7
    fn_model_out = "./trained_model13.pickle"
    fn_data_out = "BBS_dataset.pickle"

    
if True:
    pickle.dump({'train_x':X_train,
                 'train_y':Y_train,
                 'valid_x':X_test,
                 'valid_y':Y_test,
                 'label' : label}, open(fn_data_out, "wb"))
else:
    dataset = pickle.load(open("BBS_dataset.pickle", "rb"))
    X_train = dataset['train_x']
    Y_train = dataset['train_y']
    X_test = dataset['valid_x']
    Y_test = dataset['valid_y']
    
    
model = RandomForestClassifier(n_estimators=ntree, max_depth=max_depth)

model.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score
pred = model.predict(X_test)
print("정확도 :{0:.3f}".format(accuracy_score(Y_test, pred)))

pickle.dump(model, open(fn_model_out, "wb"))

정확도 :0.901


In [15]:
trees = model.estimators_

dt = trees[0]

# Feature trackback

In [17]:
farr = dt.tree_.feature[dt.tree_.feature > 0]

In [20]:
features = [ff+f"_{i}" for i in range(nframe) for ff in sub.dtype.names if ff not in "frame" ]
features = np.array(features)

In [21]:
features[farr]

array(['yhead_3', 'xr_elbow_9', 'xr_elbow_2', 'yr_foot_3',
       'yl_shoulder_4', 'xneck_7', 'xl_shoulder_0', 'xr_shoulder_1',
       'xr_shoulder_4', 'yl_elbow_6', 'yl_foot_1', 'xr_foot_3',
       'yr_shoulder_9', 'xl_hip_6', 'xhead_5', 'xl_hip_0', 'yl_foot_6',
       'xr_elbow_6', 'ypelvis_7', 'xl_elbow_1', 'yr_foot_7', 'xl_foot_9',
       'xl_foot_2', 'xhead_2', 'xr_foot_4', 'yr_elbow_4', 'xr_knee_4',
       'xr_knee_5', 'yhead_6', 'xl_hand_6', 'xl_hip_2', 'yr_hand_3',
       'xneck_0', 'xl_shoulder_4', 'yr_elbow_7', 'yl_foot_0',
       'xr_shoulder_3', 'xl_foot_5', 'xr_hand_2', 'yl_elbow_5',
       'yr_elbow_2', 'xr_foot_2', 'yl_knee_3', 'xl_elbow_4', 'xpelvis_0',
       'yr_elbow_3', 'yl_foot_1', 'yl_knee_9', 'yhead_0', 'yr_hand_9',
       'yl_knee_5', 'xl_shoulder_6', 'yl_shoulder_6', 'xl_foot_1',
       'yl_elbow_2', 'yhead_0', 'xr_elbow_6'], dtype='<U13')

# Ablation test

# Test with new data

In [208]:
import kinect_utils as ku

import pickle
point_pickle = "/home/hoseung/Work/Kinect/pykinect_BBS/G1/000/BT/bodytracking_data.pickle"
klist = pickle.load(open(point_pickle, "rb"))

scene = ku.kinect2mobile_direct(klist)

nframe = 10 
shift=0
sub = smoothed_frame_N(scene, nframe=nframe, shift=j)
rav_sub = ravel_rec(sub)[np.newaxis, :]
pickle.dump(rav_sub, open("rav_example.pickle", "wb"))

In [206]:
pwd

'/home/hoseung/Work/fhenrf/pose'

In [193]:
model.predict(rav_sub[np.newaxis,:])

array([2])

## Data test

In [5]:
## 데이터가 이상하게 잘린 친구들 

In [10]:
import numpy as np
import os

er = '/home/hoseung/Work/data/BBS/npy_a/11/4/204/a_204_11_4_0.npy'
img_dir =os.listdir('/home/hoseung/Work/data/BBS/RGB/11/4/204')
zz = np.load(er)
print('npy :', len(zz['frame']))
print('img :', len(img_dir))

npy : 192
img : 192


In [11]:
er2 = '/home/hoseung/Work/data/BBS/npy_a/11/4/379/a_379_11_4_0.npy'

img_dir2 =os.listdir('/home/hoseung/Work/data/BBS/RGB/11/4/379')
zz = np.load(er2)
print('npy :', len(zz['frame']))
print('img :', len(img_dir2))

npy : 198
img : 198


In [12]:
er3 = '/home/hoseung/Work/data/BBS/npy_a/10/3/378/a_378_10_3_0.npy'
img_dir3 =os.listdir('/home/hoseung/Work/data/BBS/RGB/10/3/378')
zz = np.load(er3)
print('npy :', len(zz['frame']))
print('img :', len(img_dir3))

npy : 769
img : 769
